# 🚀 Snowflake AISQL for CPG Promotion Management
#### 📊 Unlocking ROI from Promotions with Next-Gen Insights

---

## 🎯 Introduction

## 💡 Why Promotion Management Needs a Rethink

Consumer Packaged Goods (CPG) brands spend **up to 20% of gross revenue on trade promotions**—but over **50% of promotions fail to deliver positive ROI**.

**Why?**
- 💡 Most root causes are buried in **unstructured data**: shelf photos, field rep notes, and customer feedback from social posts, surveys, and reviews.
- 💡 Siloed, delayed analysis leads to **wasted spend, compliance misses, and lost sales**.

### 🔍 The Opportunity with Snowflake AISQL

Snowflake’s **AISQL** opens up new ways to extract, aggregate, and act on insights hidden in text, images, and external feeds, all from your data cloud.


## ✅ In this Notebook we will Extract Insights from Customer Feedback

* Create synthetic tables for promotions and feedback.
* Filter feedback to keep only promotion‑related comments using `AI_FILTER`.
* Classify feedback into categories (praise, complaint, suggestion or spam) with `AI_CLASSIFY`.
* Analyse sentiment across specific aspects (price, quality, brand perception, redemption and shipping) using `AI_SENTIMENT`.
* Summarise feedback per promotion with `AI_AGG`.
* Extract targeted answers from feedback using `SNOWFLAKE.CORTEX.EXTRACT_ANSWER`.
* Generate improved promotional copy with `AI_COMPLETE`.
* Classify promotions by discount type using `AI_CLASSIFY`.

Cells prefixed with `--` contain SQL statements that can be run in Snowflake
Notebooks (Warehouse Runtime).  Markdown cells explain the purpose of each step.


## 🚀 1  Create tables and insert synthetic data

To make this demo feel realistic, we simulate a variety of **promotion‑related data sources** for a consumer‑goods retailer.  We will create and populate four tables:

* 💡 **PROMOTIONS** – each row is a promotional campaign with descriptive fields like name, product category, channel (email, app, Instagram), discount type (percentage, BOGO, bundle, clearance, loyalty) and start/end dates.
* 💡 **FEEDBACK** – free‑form customer comments linked to a promotion.  These could come from post‑purchase surveys, app reviews or email responses.  They include praise, complaints, suggestions and spam.
* 💡 **SOCIAL_POSTS** – public social media posts captured by listening tools.  These include remarks about our promotions **and competitor promotions**, as well as unrelated chatter.
* 💡 **CALLER_NOTES** – excerpts from call center notes capturing longer, more nuanced feedback.

We load these tables with **synthetic data** that intentionally varies tone, mentions coupon codes and refers to competitor offers.  This variety allows us to showcase the capabilities of each AISQL function in a retail context.


In [ ]:
-- Create tables
CREATE OR REPLACE TABLE PROMOTIONS (
  promo_id         INT AUTOINCREMENT,
  name             STRING,
  description      STRING,
  product_category STRING,
  channel          STRING,
  discount_type    STRING,
  discount_value   NUMERIC,
  start_date       DATE,
  end_date         DATE
);

CREATE OR REPLACE TABLE FEEDBACK (
  feedback_id    INT AUTOINCREMENT,
  promo_id       INT,
  channel        STRING,
  feedback_txt   STRING,
  created_at     TIMESTAMP
);


In [ ]:
-- Insert synthetic promotions
INSERT INTO PROMOTIONS (name, description, product_category, channel, discount_type, discount_value, start_date, end_date) VALUES
('Organic Coffee Sale', '20% off organic coffee', 'beverages', 'email', 'percent', 20, '2025-07-01', '2025-07-15'),
('Laundry Pods BOGO', 'Buy 2 Get 1 Free on eco-friendly laundry pods', 'household', 'instagram', 'bogo', NULL, '2025-07-05', '2025-07-20'),
('Skin Care Clearance', '50% off summer skin-care sets', 'personal care', 'push', 'clearance', 50, '2025-07-10', '2025-07-30'),
('Loyalty Bonus Points', 'Earn 500 bonus points on purchases over $50', 'loyalty', 'app', 'loyalty', 500, '2025-07-01', '2025-07-31'),
('Snack Bundle Deal', 'Buy 3 snacks and get 20% off fruit packs', 'snacks', 'website', 'bundle', 20, '2025-07-12', '2025-07-25');

-- Insert synthetic feedback
INSERT INTO FEEDBACK (promo_id, channel, feedback_txt, created_at) VALUES
-- Feedback for Organic Coffee Sale (promo_id=1)
(1, 'social', 'Loved the 20% coffee discount! Great taste, but the code didn''t work at first.', CURRENT_TIMESTAMP),
(1, 'email',  'Is the 20% coffee discount valid in store as well?', CURRENT_TIMESTAMP),
(1, 'social', 'Had to call support because the coffee coupon code "COFFEE20" kept failing.', CURRENT_TIMESTAMP),
(1, 'social', 'The coffee sale is decent but competitor''s 25% off is better.', CURRENT_TIMESTAMP),
(1, 'app',   'The organic coffee tastes great but the bag was half empty.', CURRENT_TIMESTAMP),
(1, 'social', 'Would love a bigger discount on the coffee sale next time.', CURRENT_TIMESTAMP),
(1, 'social', 'The 20% coffee discount helped me try new flavors; loved it!', CURRENT_TIMESTAMP),
(1, 'support', 'Customer support quickly fixed my COFFEE20 code issue.', CURRENT_TIMESTAMP),

-- Feedback for Laundry Pods BOGO (promo_id=2)
(2, 'social', 'Your BOGO on laundry pods saved me money. Please extend the sale.', CURRENT_TIMESTAMP),
(2, 'social', 'The buy-one-get-one laundry promo didn''t apply at checkout. Very frustrating.', CURRENT_TIMESTAMP),
(2, 'instagram', 'Thanks for the eco-friendly laundry pods BOGO! Smells amazing.', CURRENT_TIMESTAMP),
(2, 'email', 'Happy with BOGO laundry pods but shipping took too long.', CURRENT_TIMESTAMP),
(2, 'support', 'Customer service helped me redeem the BOGO after the code didn''t work.', CURRENT_TIMESTAMP),
(2, 'social', 'The pods leave residue; nothing to do with promotions though.', CURRENT_TIMESTAMP),
(2, 'social', 'Wish the BOGO applied to the lavender-scented pods too.', CURRENT_TIMESTAMP),
(2, 'app', 'App kept freezing when I tried to apply the BOGO; please fix.', CURRENT_TIMESTAMP),
(2, 'social', 'Laundry pod fragrance too strong; no BOGO could make me buy again.', CURRENT_TIMESTAMP),

-- Feedback for Skin Care Clearance (promo_id=3)
(3, 'push', '50% off skin care sets is an amazing deal! Can''t wait to try the brand''s new moisturizer.', CURRENT_TIMESTAMP),
(3, 'push', 'The clearance sale had limited stock and shipping was slow.', CURRENT_TIMESTAMP),
(3, 'push', 'Stop sending me push notifications about skin care deals.', CURRENT_TIMESTAMP),
(3, 'push', 'Love the summer clearance sale! Please restock sunscreen.', CURRENT_TIMESTAMP),
(3, 'push', 'The skin care discount is great, but packaging arrived damaged.', CURRENT_TIMESTAMP),
(3, 'email', 'Why is the skin care sale only online? Love the products though.', CURRENT_TIMESTAMP),
(3, 'social', 'The moisturizer is oily; not happy even at 50% off.', CURRENT_TIMESTAMP),
(3, 'email', 'Thanks for the 50% off; I purchased two sets and got an extra sample.', CURRENT_TIMESTAMP),

-- Feedback for Loyalty Bonus Points (promo_id=4)
(4, 'app', 'Earned 500 bonus points! Love the loyalty program.', CURRENT_TIMESTAMP),
(4, 'app', 'Why is the app so buggy when I try to get my bonus points?', CURRENT_TIMESTAMP),
(4, 'app', 'The loyalty program is a scam, my points expired too soon.', CURRENT_TIMESTAMP),
(4, 'social', 'I like the loyalty program but the bonus points expired too soon.', CURRENT_TIMESTAMP),
(4, 'email', 'Earning bonus points feels rewarding; love the loyalty program.', CURRENT_TIMESTAMP),
(4, 'social', 'My bonus points didn''t show up; had to contact support.', CURRENT_TIMESTAMP),
(4, 'email', 'Can I transfer my loyalty points to a friend?', CURRENT_TIMESTAMP),

-- Feedback for Snack Bundle Deal (promo_id=5)
(5, 'website', 'The snack bundle discount is confusing. How do I redeem it?', CURRENT_TIMESTAMP),
(5, 'website', 'Great snack bundle! My kids loved the variety.', CURRENT_TIMESTAMP),
(5, 'website', 'Your snack bundle discount should include vegan options.', CURRENT_TIMESTAMP),
(5, 'social', 'I got the snack bundle but shipping cost was more than the discount.', CURRENT_TIMESTAMP),
(5, 'app', 'The snack bundle offer encourages me to buy more fruits; love it!', CURRENT_TIMESTAMP),
(5, 'social', 'Bundle discount should combine with other coupons.', CURRENT_TIMESTAMP),
(5, 'email', 'Is the snack bundle deal available in store? I prefer shopping there.', CURRENT_TIMESTAMP),

-- General or spam feedback (promo_id=NULL)
(NULL, 'social', 'Spam message: Click here for free gift cards.', CURRENT_TIMESTAMP),
(NULL, 'social', 'This is irrelevant spam about real estate.', CURRENT_TIMESTAMP),
(NULL, 'social', 'Will there be a discount on pet food soon?', CURRENT_TIMESTAMP),
(NULL, 'social', 'I don''t care about promotions, just show me new products.', CURRENT_TIMESTAMP),
(NULL, 'social', 'The packaging on my order was damaged even though I used no promo.', CURRENT_TIMESTAMP),
(NULL, 'email', 'Great customer service, but I wish for more organic options.', CURRENT_TIMESTAMP),
(NULL, 'app', 'The app is slow and buggy; please fix.', CURRENT_TIMESTAMP),
(NULL, 'social', 'Competitor BrandY has free shipping on snacks; consider matching.', CURRENT_TIMESTAMP),
(NULL, 'social', 'Just leaving a review for your new almond milk—no promo used.', CURRENT_TIMESTAMP),
(NULL, 'email', 'My order arrived late but customer service compensated me; no promo involvement.', CURRENT_TIMESTAMP),
(NULL, 'app', 'I can never find nutrition info easily in the app.', CURRENT_TIMESTAMP),
(NULL, 'social', 'Competitor Z has a free gift with purchase—curious if you plan a similar promo.', CURRENT_TIMESTAMP),
(NULL, 'social', 'Stop spamming my inbox with promotions; I''m not interested.', CURRENT_TIMESTAMP),
(NULL, 'social', 'The website search is slow; please improve user experience.', CURRENT_TIMESTAMP),
(NULL, 'social', 'Please bring back the free shipping promotion from last year.', CURRENT_TIMESTAMP),
(NULL, 'app', 'Love the new dark mode in the app.', CURRENT_TIMESTAMP);

In [ ]:
-- Create social_posts table and insert synthetic posts
CREATE OR REPLACE TABLE SOCIAL_POSTS (
  post_id INT AUTOINCREMENT,
  channel STRING,
  content STRING,
  created_at TIMESTAMP
);

INSERT INTO SOCIAL_POSTS (channel, content, created_at) VALUES
('social', 'Loving the 20% off organic coffee! Anyone else tried it?', CURRENT_TIMESTAMP),
('social', 'The BOGO laundry pods deal saved me so much money this week.', CURRENT_TIMESTAMP),
('social', 'Skin care clearance sale is legit 50% off – hurry!', CURRENT_TIMESTAMP),
('social', 'Loved the 20% coffee sale but shipping costs were high.', CURRENT_TIMESTAMP),
('social', 'BrandX just announced 25% off coffee pods – should I switch?', CURRENT_TIMESTAMP),
('social', 'Saw a competitor offering buy one snack, get one free on chips.', CURRENT_TIMESTAMP),
('social', 'Competitor coffee sale beats this by a mile.', CURRENT_TIMESTAMP),
('social', 'Competitor BrandY offers free shipping on all snack orders this week.', CURRENT_TIMESTAMP),
('social', 'Just bought a new espresso machine. Any bean recommendations?', CURRENT_TIMESTAMP),
('social', 'Just tried the new laundry pods; the scent is amazing!', CURRENT_TIMESTAMP),
('social', 'Why is the loyalty app so slow during checkout?', CURRENT_TIMESTAMP),
('social', 'Coffee sale is great but the coupon code didn''t work on the mobile site.', CURRENT_TIMESTAMP),
('social', 'I prefer competitor BrandZ’s coffee; their discount is better.', CURRENT_TIMESTAMP),
('social', 'Anyone know if the laundry pod BOGO applies in-store?', CURRENT_TIMESTAMP),
('social', 'The clearance sale on skin care items made my day!', CURRENT_TIMESTAMP),
('social', 'Why so many clearance emails? I got three today.', CURRENT_TIMESTAMP),
('social', 'Just unboxed my snack bundle—kudos on the sustainable packaging.', CURRENT_TIMESTAMP),
('social', 'Competitor Y is offering 30% off snacks; tempted to switch.', CURRENT_TIMESTAMP),
('social', 'Love the bonus points on purchases over $50; I got 600 points last week!', CURRENT_TIMESTAMP),
('social', 'How do I redeem my loyalty bonus points at checkout?', CURRENT_TIMESTAMP),
('social', 'Shipping took ages for my last order. No discount could make up for it.', CURRENT_TIMESTAMP),
('social', 'My friend got a coupon but I didn’t—how do you qualify?', CURRENT_TIMESTAMP),
('social', 'I spotted your coffee promotion on TikTok; looks fun!', CURRENT_TIMESTAMP),
('social', 'The new app update crashed while I was browsing deals.', CURRENT_TIMESTAMP),
('social', 'Competitor Q just launched a BOGO on energy drinks.', CURRENT_TIMESTAMP),
('social', 'Why isn’t there a promotion on pet food?', CURRENT_TIMESTAMP),
('social', 'Got free shipping on my snack bundle—thanks!', CURRENT_TIMESTAMP),
('social', 'Coffee pods arrived broken. Really disappointed.', CURRENT_TIMESTAMP),
('social', 'Looking forward to next week’s promotion—any hints?', CURRENT_TIMESTAMP),
('social', 'Just saw an influencer talk about your skin care sale; had to buy.', CURRENT_TIMESTAMP),
('social', 'Competitor’s loyalty program gives triple points this month.', CURRENT_TIMESTAMP),
('social', 'No discount but the quality of the coffee is worth it.', CURRENT_TIMESTAMP),
('social', 'BOGO deals always make me spend more than I planned!', CURRENT_TIMESTAMP),
('social', 'Wishlist: bundle discount on cleaning products.', CURRENT_TIMESTAMP),
('social', 'The buy-more-save-more promotion on snacks was confusing.', CURRENT_TIMESTAMP),
('social', 'Just received a push notification for 20% off coffee pods; nice!', CURRENT_TIMESTAMP),
('social', 'Shipping fee negated my savings from the coupon.', CURRENT_TIMESTAMP),
('social', 'Competitor has free shipping plus 20% off this weekend.', CURRENT_TIMESTAMP),
('social', 'Why not run promotions on organic teas?', CURRENT_TIMESTAMP),
('social', 'I don’t care about promotions; I want faster restocks.', CURRENT_TIMESTAMP),
('social', 'Just tried the new loyalty app; smooth and responsive now!', CURRENT_TIMESTAMP),
('social', 'My order came with a thank-you note—nice touch.', CURRENT_TIMESTAMP),
('social', 'First time hearing about your brand from a promotion; impressed.', CURRENT_TIMESTAMP),
('social', 'Competitor’s promotion has fewer restrictions; easier to redeem.', CURRENT_TIMESTAMP),
('social', 'More vegan options, please! Promotions or not, that’s important.', CURRENT_TIMESTAMP),
('social', 'Waited on hold 20 minutes for support about a promo code.', CURRENT_TIMESTAMP),
('social', 'Is the BOGO still active? I can’t find it on the site.', CURRENT_TIMESTAMP),
('social', 'Love your environmental messaging in the coffee sale campaign.', CURRENT_TIMESTAMP),
('social', 'Got an extra sample in my skin care order; love freebies.', CURRENT_TIMESTAMP),
('social', 'Competitor brand''s sale ended early. Hope yours lasts longer.', CURRENT_TIMESTAMP);

-- Create caller_notes table and insert synthetic transcripts
CREATE OR REPLACE TABLE CALLER_NOTES (
  call_id INT AUTOINCREMENT,
  note STRING
);

INSERT INTO CALLER_NOTES (note) VALUES
('Customer called about the coffee discount code COFFEE20 not working at checkout and asked for assistance.'),
('Caller complained that the laundry pods promotion did not apply and shipping was delayed.'),
('Customer praised the skin care clearance but mentioned damaged packaging and slow delivery.'),
('Caller asked whether competitor BrandX has a better coffee promotion than ours.'),
('Customer expressed confusion about the snack bundle redemption process and asked for clarification.'),
('Random call: a wrong number, unrelated to promotions.'),
('Customer liked the coffee sale but reported that the bag arrived half empty.'),
('Caller praised the loyalty program but was upset that bonus points expired before they could use them.'),
('Customer mentioned that competitor BrandY offers free shipping on snacks and asked if we plan to match it.'),
('Caller asked how to combine the snack bundle discount with free shipping.'),
('Customer praised the organic coffee but said the bag was only half full.'),
('Customer reported that the loyalty app crashed during checkout.'),
('Caller wanted to know when the next skin care sale will be.'),
('Customer complained about late delivery despite paying for expedited shipping.'),
('Customer asked if COFFEE20 works on decaf beans.'),
('Caller was satisfied with support helping them redeem the laundry pods BOGO.'),
('Customer wanted to exchange damaged laundry pods received during the promotion.'),
('Caller mentioned competitor BrandX offering a better BOGO on laundry pods.'),
('Customer was annoyed that the coupon was only valid online.'),
('Caller said the push notifications are too frequent and distracting.'),
('Customer noted that the discount didn’t apply due to product exclusions.'),
('Caller appreciated the free sample included with their skin care purchase.'),
('Customer asked whether loyalty points can be used with promotions.'),
('Customer said the coffee tasted burnt and wanted a refund.'),
('Caller asked about vegan options in upcoming promotions.'),
('Customer reported receiving duplicate orders by mistake during the sale.'),
('Caller mentioned their friend got a larger discount for the same product.'),
('Customer thanked the team for extending the promotion period.'),
('Caller wanted to know if competitor’s free shipping would be matched.'),
('Customer suggested a “mix and match” bundle promotion.'),
('Caller was confused about how points are calculated in the loyalty program.'),
('Customer complained about the lack of promotions on baby products.'),
('Caller asked if their expired points could be reinstated.'),
('Customer wanted to know when the next BOGO will be.'),
('Caller loved the eco-friendly packaging but wanted more discount.'),
('Customer reported that the discount code leaked before the official launch.'),
('Caller complained that the website crashed on sale day.'),
('Customer asked if there was a limit on the number of promo codes per order.'),
('Caller inquired about combining the clearance sale with loyalty points.'),
('Customer said competitor offers a gift-with-purchase for the same product.'),
('Caller asked if shipping fees are waived during promotions.'),
('Customer noted that the app still showed an expired promotion.'),
('Caller appreciated the clear communication about promotion start and end dates.'),
('Customer said the skin care product caused irritation and sought a return.'),
('Caller asked for a price match with competitor BrandZ’s promotion.'),
('Customer shared that they prefer shopping in-store during promotions for instant gratification.'),
('Caller asked how to opt out of promotional emails.'),
('Customer wanted information about upcoming holiday promotions.'),
('Caller said the checkout cart removed the discount when adding another item.'),
('Customer shared positive feedback about the customer service representative who resolved their promo issue.');


## 📊 2  Filter promotion‑related feedback with `AI_FILTER`

Our customers speak about many topics – not all of them are relevant to marketing promotions.  To focus our analysis, we use **AI_FILTER** to answer the question:

> *“Does this comment mention a promotion, coupon, discount code, deal, offer, sale or bundle?”*

The function returns `TRUE` or `FALSE` for each feedback record.  We create a view **PROMO_FEEDBACK** containing only rows that mention promotions.  Inspecting this view helps us verify that irrelevant comments (e.g., requests for new products) have been removed.


In [ ]:
-- Create a view of promotion-related feedback using AI_FILTER
CREATE OR REPLACE VIEW PROMO_FEEDBACK AS
SELECT *
FROM FEEDBACK
WHERE AI_FILTER(
  PROMPT('In the following text, does the customer mention a promotion, coupon, discount code, deal, offer, sale or bundle? Text: {0}', feedback_txt)
);

-- Inspect the filtered promo feedback
SELECT * FROM PROMO_FEEDBACK ORDER BY feedback_id;

## 🎯 3  Remove spam and unrelated comments using `AI_FILTER`

Even within promotion‑related feedback, there may be **spam** or unrelated content that can skew our analysis.  We ask the model another question:

> *“Is this comment spam, unrelated to retail promotions, or does it mention real estate, gift cards or other scams?”*

Comments returning `TRUE` to this predicate are excluded in a cleaned view **PROMO_FEEDBACK_CLEAN**.  This step ensures that downstream classifications and sentiment analysis focus on genuine customer feedback.


In [ ]:
-- Create a view that removes obvious spam or irrelevant comments
CREATE OR REPLACE VIEW PROMO_FEEDBACK_CLEAN AS
SELECT *
FROM PROMO_FEEDBACK
WHERE NOT AI_FILTER(
  PROMPT('Return TRUE if the comment is spam, unrelated to retail promotions or mentions real estate, gift cards or other scams. Comment: {0}', feedback_txt)
);

-- Inspect the cleaned promo feedback
SELECT *
FROM PROMO_FEEDBACK_CLEAN
ORDER BY feedback_id;

## 💡 4  Classify feedback categories with `AI_CLASSIFY`

Not all feedback is created equal.  Classifying comments into categories enables targeted action.  We use **AI_CLASSIFY** to label each cleaned comment as one or more of **praise**, **complaint**, **suggestion** or **spam**.  The `output_mode` is set to *multi*, allowing a single comment to be both a complaint and a suggestion.  This categorisation helps marketing teams quickly identify praise to amplify and complaints to address.


In [ ]:
SELECT feedback_id,
       AI_CLASSIFY(
         feedback_txt,
         ['praise','complaint','suggestion','spam'],
         {
           'task_description': 'Categorize the retail promotion feedback as praise, complaint, suggestion or spam. Some comments may belong to multiple categories.',
           'output_mode': 'multi'
         }
       ):labels AS category,
       feedback_txt
FROM PROMO_FEEDBACK_CLEAN
ORDER BY feedback_id;

## 🔍 5  Analyse sentiment across key aspects with `AI_SENTIMENT`

Customers may love the product but hate the checkout experience.  To capture this nuance, **AI_SENTIMENT** provides both an **overall sentiment** and aspect‑specific sentiments.  We evaluate five aspects relevant to promotions in retail:

* 💡 **price** – Was the discount compelling?
* 💡 **quality** – Did the product meet expectations?
* 💡 **brand** – How do customers feel about the brand?
* 💡 **redemption** – Did coupon codes work smoothly?
* 💡 **shipping** – Were delivery times and logistics acceptable?

The returned JSON object tells us whether each aspect is positive, negative, neutral, mixed or unknown, enabling data‑driven prioritisation of improvements.


In [ ]:
SELECT feedback_id,
       AI_SENTIMENT(
         feedback_txt,
         ['price','quality','brand','redemption','shipping']
       ) AS sentiment,
       feedback_txt
FROM PROMO_FEEDBACK_CLEAN
ORDER BY feedback_id;

## ✅ 6  Summarise feedback per promotion using `AI_AGG`

Reading through individual comments doesn’t scale.  With **AI_AGG**, we summarise dozens of feedback rows into a concise paragraph per promotion.  The task description instructs the model to highlight key positives and negatives in under 120 words, focusing on price, quality, redemption and shipping.  This provides decision‑makers with an executive summary of each campaign’s reception.


In [ ]:
SELECT promo_id,
       AI_AGG(feedback_txt,
              'Summarize key positives and negatives in <=120 words for the promotion feedback provided. Focus on price, quality, redemption and shipping experiences.'
       ) AS summary
FROM PROMO_FEEDBACK_CLEAN
GROUP BY promo_id
ORDER BY promo_id;

## 🚀 7  Extract coupon codes and other details with `SNOWFLAKE.CORTEX.EXTRACT_ANSWER`

Sometimes we need to pull out a specific piece of information from unstructured text.  **EXTRACT_ANSWER** allows us to ask targeted questions like:

> *“What coupon or discount code is mentioned in this feedback?”*

The function returns the answer as a string.  You could extend this pattern to extract quantities purchased, competitor names or other facts, turning messy comments into structured columns.


In [ ]:
SELECT feedback_id,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(
         feedback_txt,
         'What coupon or discount code is mentioned in this feedback?'
       ) AS code_mentioned,
       feedback_txt
FROM PROMO_FEEDBACK_CLEAN
ORDER BY feedback_id;

## 📊 8  Generate improved promotional copy with `AI_COMPLETE`

After analysing feedback, marketers might decide to refresh a campaign.  **AI_COMPLETE** leverages Snowflake’s LLM to draft new marketing copy directly from SQL.  In this demo we ask for a playful, 25‑word Instagram caption promoting our eco‑friendly laundry pods BOGO offer.  You can experiment with tone (`temperature`), length (`max_tokens`) and even output structured JSON for taglines or call‑to‑action suggestions.


In [ ]:
SELECT AI_COMPLETE(
  model  => 'claude-4-sonnet',
  prompt => 'Write a concise 25-word Instagram caption promoting our eco-friendly laundry pods BOGO. Use a playful tone and highlight sustainability.',
  model_parameters => {
    'temperature': 0.7,
    'max_tokens': 60
  }
);


## 🎯 9  Detect competitor promotions with `AI_FILTER`

Competitor activity can quickly erode the impact of your campaign.  By scanning **SOCIAL_POSTS** with **AI_FILTER**, we flag posts where customers mention a **competitor promotion** for coffee or snacks.  This proactive monitoring helps the marketing team decide whether to respond, match or differentiate against rival offers.


In [ ]:
SELECT post_id, content
FROM SOCIAL_POSTS
WHERE AI_FILTER(
  PROMPT('Does this social media post explicitly mention any specific competitor companies, brands, or products by name? Content: {0}', content)
);

## 💡 10  Aggregate themes across call notes with `AI_AGG`

Longer form feedback from call center notes often reveal recurring pain points that short comments miss.  We use **AI_AGG** on **CALL_NOTES** with a task description:

> *“Identify common issues across these notes and summarise as bullet points.”*

The result groups together themes such as “discount code failures” or “shipping delays” so that operations teams can prioritise fixes.


In [ ]:
SELECT AI_AGG(
        'Field note: ' || note,
        'Identify common issues across these notes and summarise as bullet points'
    ) AS aggregated_themes
FROM CALLER_NOTES;

In [ ]:
SELECT AI_AGG(
    'Field note: ' || note,
    'Analyze these customer service notes and categorize the main issues into themes. For each theme, provide: 1) A brief description of the problem, 2) How frequently it occurs, and 3) The potential business impact. Focus on operational issues, technical problems, competitive concerns, and customer experience gaps that require attention.'
) AS aggregated_themes
FROM CALLER_NOTES;

## 🔍 11  Classify promotions by discount type with `AI_CLASSIFY`

Here, we classify the **PROMOTIONS** table itself.  Using **AI_CLASSIFY**, we identify whether each campaign is a **percentage discount**, **BOGO**, **bundle**, **clearance** or **loyalty** offer.  This step illustrates that AISQL functions work on structured descriptions too, making it easier to catalogue and search campaigns.


In [ ]:
SELECT promo_id,
       AI_CLASSIFY(
         description,
         ['percentage discount','bogo','bundle','clearance','loyalty'],
         {
           'task_description': 'Identify the type of retail promotion from the description. Choose from percentage discount, bogo, bundle, clearance or loyalty.',
           'output_mode': 'single'
         }
       ):labels AS promo_type,
       description
FROM PROMOTIONS
ORDER BY promo_id;


# 🚀 Conclusion: Turning Promo Data into Profitable Action

By leveraging **Snowflake Cortex AISQL** functions directly inside a notebook, we've built a complete **promotion-management workflow** that covers:

- 🧹  Cleaning and classifying feedback 
- 📊  Analyzing sentiment and summarizing insights
- 🕵️‍♀️  Detecting competitor activity  
- ✍️  Generating fresh marketing copy

This demo illustrates how **retail and consumer goods teams** can harness the power of **large language models without ever leaving SQL**.  

- ✅ Unlock hidden insights from images, field notes, and customer feedback —**without data engineering pipeline bottlenecks** 
- ✅ Boost promotion ROI, compliance, and agility 
- ✅ Move from reactive “post-mortem” to proactive, real-time promo optimization

---

## Next Steps:
- 💡 Try these queries on your own data (or with the dummy set in this notebook)
- 💡 Pilot a live promotion monitoring use case with your Snowflake team
- 💡 For advanced AISQL adoption, reach out to your Snowflake rep or visit [Snowflake AISQL documentation](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql)
